# 10 - Agentic RAG (Autonomous Agents)

**Complexity:** ⭐⭐⭐⭐⭐

**Use Cases:** Multi-step reasoning, BI dashboards, complex decision-making

**Key Features:**
- ReAct agent (Reasoning + Acting)
- Multiple tools (retriever, calculator, web search)
- Autonomous planning and execution
- Multi-step reasoning

**Example:**
```
Query: "If I have 10K docs and process 1M tokens/day, should I use OpenAI or HF?"

Agent:
1. Thought: Need cost calculation
   Action: Calculator → Estimate costs
2. Thought: Need comparison info
   Action: Knowledge Base → Retrieve comparison
3. Thought: Can now recommend
   Final Answer: "HuggingFace is better because..."
```

In [ ]:
import sys
sys.path.append('../..')

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from shared.config import OPENAI_VECTOR_STORE_PATH, DEFAULT_MODEL
from shared.utils import load_vector_store, print_section_header, format_docs
from shared.prompts import REACT_AGENT_PROMPT

print_section_header("Setup: Agentic RAG")

embeddings = OpenAIEmbeddings()
vectorstore = load_vector_store(OPENAI_VECTOR_STORE_PATH, embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 4})
llm = ChatOpenAI(model=DEFAULT_MODEL, temperature=0)

print("✅ Setup complete!")

## 2. Create Tools for Agent

In [ ]:
from langchain.agents import Tool
from langchain.chains import LLMMathChain

print_section_header("Agent Tools")

# Tool 1: Retriever
def retriever_tool_func(query: str) -> str:
    """Search LangChain RAG knowledge base."""
    docs = retriever.invoke(query)
    return format_docs(docs[:3])

retriever_tool = Tool(
    name="Knowledge_Base",
    func=retriever_tool_func,
    description="""Search LangChain RAG knowledge base. 
    Use for questions about RAG, embeddings, retrievers, LangChain."""
)

# Tool 2: Calculator
llm_math = LLMMathChain.from_llm(llm)
calculator_tool = Tool(
    name="Calculator",
    func=llm_math.run,
    description="""Useful for math calculations. 
    Input should be a math expression."""
)

# Tool 3: Web Search (if available)
try:
    from langchain_community.tools import DuckDuckGoSearchResults
    web_search_tool = Tool(
        name="Web_Search",
        func=DuckDuckGoSearchResults(num_results=2).invoke,
        description="""Search the web for current info. 
        Use for queries outside knowledge base."""
    )
    tools = [retriever_tool, calculator_tool, web_search_tool]
    print("✓ Created 3 tools (retriever, calculator, web)")
except:
    tools = [retriever_tool, calculator_tool]
    print("✓ Created 2 tools (retriever, calculator)")

for tool in tools:
    print(f"  - {tool.name}")

## 3. Create ReAct Agent

In [ ]:
from langchain.agents import AgentExecutor, create_react_agent

print_section_header("ReAct Agent")

# Create agent
agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=REACT_AGENT_PROMPT
)

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    max_iterations=5,
    handle_parsing_errors=True
)

print("✓ ReAct agent created")
print("  - Max iterations: 5")
print("  - Verbose mode enabled")

## 4. Test Agent - Simple Query

In [ ]:
print_section_header("Test 1: Simple Query")

query = "What is a vector store?"
print(f"Query: '{query}'\n")
print("=" * 80)

response = agent_executor.invoke({"input": query})
print("\n" + "=" * 80)
print(f"\nFinal Answer: {response['output']}")

## 5. Test Agent - Multi-Step Reasoning

In [ ]:
print_section_header("Test 2: Multi-Step")

query = """If embedding 100,000 documents costs $2 with OpenAI,
what's the cost per document? Then tell me if HuggingFace is cheaper."""

print(f"Query: {query}\n")
print("=" * 80)

response = agent_executor.invoke({"input": query})
print("\n" + "=" * 80)
print(f"\nFinal Answer: {response['output']}")

print("\n💡 Agent used multiple tools to solve this!")

## 6. Agent with Memory

In [ ]:
from langchain.agents import AgentType, initialize_agent
from langchain.memory import ConversationBufferMemory

print_section_header("Conversational Agent")

memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

conversational_agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
    memory=memory,
    verbose=False,  # Less verbose for readability
    max_iterations=5
)

print("✓ Conversational agent created\n")

# Multi-turn conversation
print("User: What are the embedding dimensions?\n")
r1 = conversational_agent.invoke({"input": "What are the embedding dimensions?"})
print(f"Agent: {r1['output'][:150]}...\n")

print("\nUser: Which one is cheaper?\n")
r2 = conversational_agent.invoke({"input": "Which one is cheaper?"})
print(f"Agent: {r2['output'][:150]}...")

print("\n✅ Agent maintained conversation context!")

## Summary

**Flow:**
```
Query → Agent Reasoning Loop:
  1. Think about what to do
  2. Select tool to use
  3. Execute tool
  4. Observe result
  5. Repeat until answer found
→ Final Answer
```

**Advantages:**
✅ Autonomous decision-making  
✅ Multi-step reasoning  
✅ Tool composition  
✅ Handles complex queries  
✅ Extensible (add more tools)  

**Limitations:**
- Very slow (many LLM calls)
- Expensive (reasoning tokens)
- Can get stuck in loops
- Unpredictable behavior
- Hard to debug

**When to Use:**
- Complex multi-step tasks
- Requires multiple data sources
- Research/analysis workflows
- BI dashboards

**Production Tips:**
- Set max_iterations carefully
- Monitor token usage
- Add timeout mechanisms
- Implement fallbacks
- Use LangSmith for tracing
- Test edge cases thoroughly

**Next:** [11_comparison.ipynb](11_comparison.ipynb) - Benchmark all architectures